In [63]:
import os
import multiprocessing
import tqdm
import pandas as pd
import json
import numpy as np
from keplergl import KeplerGl

os.environ['MAPBOX_API_KEY'] = "..." # Input something

INPUT_FILE = "Location History.json"
OUTPUT_FILE = "google-location-data.parquet"
OUTPUT_HTML_FILE = "kepler_gl_map.html"

In [64]:
with open(INPUT_FILE, "r") as file:
    input_json = json.load(file)

In [65]:
print("No:of points - " + str(len(input_json['locations'])))

No:of points - 132027


In [66]:
df = pd.DataFrame(input_json['locations'])

In [67]:
df["datetime"] = pd.to_datetime(df["timestampMs"], unit="ms")
df.drop(columns=['accuracy', 'velocity', 'heading', 'verticalAccuracy', 'activity', 'timestampMs'], inplace=True)
df.rename(columns={'latitudeE7': 'lat'}, inplace=True)
df.rename(columns={'longitudeE7': 'lon'}, inplace=True)
df['lat'] = df['lat'].div(10000000)
df['lon'] = df['lon'].div(10000000)

In [68]:
df.to_parquet(OUTPUT_FILE, compression='brotli')

In [69]:
# If we take all data points, the render is often slow
df_kepler = df.sample(25000)
print(df_kepler.shape[0])
df_kepler.head()

# Reformat datetime to string for JSON serialization
df_kepler['datetime'] = df_kepler['datetime'].astype(str)

132027


In [70]:
config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': df_kepler.lat.median(),
            'longitude': df_kepler.lon.median(),
            'zoom': 5
        }
    }
}

keplermap = KeplerGl(height=900, config=config)

# Add the data
keplermap.add_data(data=df_kepler, name='data1')

keplermap

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 27.4789508, 'longitude': 79.3952665, 'zo…

In [71]:
# save current map
keplermap.save_to_html(file_name=OUTPUT_HTML_FILE)

Map saved to kepler_gl_map.html!
